In [1]:
import json
import random
import torch

In [2]:
import pandas as pd

dataset = pd.read_csv("data_full_v1.csv")

In [3]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)


In [4]:
dataset['prompt'] = dataset['prompt'].str.lower()

In [5]:
dataset

,id,prompt,label,ground_truth
0,4,husband has a direct effect on wife and alarm ...,yes,1
1,7,husband has a direct effect on wife and alarm ...,no,0
2,8,husband has a direct effect on wife and alarm ...,yes,1
3,15,husband has a direct effect on wife and alarm ...,no,0
4,21,husband has a direct effect on wife and alarm ...,no,0
...,...,...,...,...
10107,31012,zuph has a direct effect on glimx. jyka has a ...,yes,1
10108,31014,zuph has a direct effect on glimx. jyka has a ...,yes,1
10109,31015,zuph has a direct effect on glimx. jyka has a ...,no,0
10110,31016,zuph has a direct effect on glimx. jyka has a ...,no,0


In [6]:
dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [7]:
from torch.utils.data import Dataset

class CladderDataset(Dataset):
    def __init__(self, cladder_data, tokenizer):
        self.cladder_data = cladder_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.cladder_data)

    def __getitem__(self, idx):
        data = self.cladder_data[idx]
        # print(data)
        prompt = data['prompt']
        label = data['ground_truth']

        # print(prompt)
        # print(label)

        # Tokenize
        encoded_input = self.tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').to("cuda")
        # print(encoded_input)
        encoded_input['labels'] = torch.tensor(label).to("cuda")

        return encoded_input


In [8]:
# from transformers import AlbertTokenizer, AlbertForSequenceClassification
# import torch
# from torch.utils.data import DataLoader
# import torch.optim as optim

# # Load tokenizer and model
# tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
# model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels = 2)

In [9]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [10]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import AlbertTokenizer as tkz
from transformers import AlbertForSequenceClassification as mpc

# Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

# run = 0
# for run in range(1):

for run in range(3):
  model_base = 'albert-base-v2'
  tokenizer = tkz.from_pretrained(model_base)
  model = mpc.from_pretrained(model_base, num_labels = 2)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  # Prepare the DataLoader
  cladder_dataset = CladderDataset(train_data, tokenizer)
  dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

  # Optimizer and Loss Function
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  loss_fn = torch.nn.CrossEntropyLoss()



  # Training Loop
  model.train()

  #change to 3

  for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  torch.save(model.state_dict(), model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

...

Epoch: 0, Loss: 0.7095731496810913
Epoch: 0, Loss: 0.7612349987030029
Epoch: 0, Loss: 0.7637986540794373
Epoch: 0, Loss: 0.8401581048965454
Epoch: 0, Loss: 0.6446523070335388
Epoch: 0, Loss: 0.765009880065918
Epoch: 0, Loss: 0.7326542735099792
Epoch: 0, Loss: 0.649212121963501
Epoch: 0, Loss: 0.6538665890693665
Epoch: 0, Loss: 0.7557125091552734
Epoch: 0, Loss: 0.6962482333183289
Epoch: 0, Loss: 0.6790494322776794
Epoch: 0, Loss: 0.7162517309188843
Epoch: 0, Loss: 0.769901692867279
Epoch: 0, Loss: 0.7448083758354187
Epoch: 0, Loss: 0.8619913458824158
Epoch: 0, Loss: 0.7573511600494385
Epoch: 0, Loss: 0.6929714679718018
Epoch: 0, Loss: 0.6456261277198792
Epoch: 0, Loss: 0.7049251794815063
Epoch: 0, Loss: 0.6318690776824951
Epoch: 0, Loss: 0.820895254611969
Epoch: 0, Loss: 0.7399256229400635
Epoch: 0, Loss: 0.6800333857536316
Epoch: 0, Loss: 0.6628275513648987
Epoch: 0, Loss: 0.7710145711898804
Epoch: 0, Loss: 0.7096896767616272
Epoch: 0, Loss: 0.7462410926818848
Epoch: 0, Loss: 0.694822

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6381915211677551
Epoch: 0, Loss: 0.6773461103439331
Epoch: 0, Loss: 0.7261770367622375
Epoch: 0, Loss: 0.8103539943695068
Epoch: 0, Loss: 0.7097475528717041
Epoch: 0, Loss: 0.7060690522193909
Epoch: 0, Loss: 0.6940104961395264
Epoch: 0, Loss: 0.7255996465682983
Epoch: 0, Loss: 0.5800290107727051
Epoch: 0, Loss: 0.7122715711593628
Epoch: 0, Loss: 0.6407536268234253
Epoch: 0, Loss: 0.8295813798904419
Epoch: 0, Loss: 0.8265315294265747
Epoch: 0, Loss: 0.8006658554077148
Epoch: 0, Loss: 0.6460821032524109
Epoch: 0, Loss: 0.6495458483695984
Epoch: 0, Loss: 0.830792248249054
Epoch: 0, Loss: 0.6973820924758911
Epoch: 0, Loss: 0.7540647983551025
Epoch: 0, Loss: 0.7883762121200562
Epoch: 0, Loss: 0.6903672218322754
Epoch: 0, Loss: 0.7147390842437744
Epoch: 0, Loss: 0.7073312997817993
Epoch: 0, Loss: 0.7124758362770081
Epoch: 0, Loss: 0.6670262813568115
Epoch: 0, Loss: 0.7734679579734802
Epoch: 0, Loss: 0.6931817531585693
Epoch: 0, Loss: 0.7238274216651917
Epoch: 0, Loss: 0.696

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7412422895431519
Epoch: 0, Loss: 0.9102133512496948
Epoch: 0, Loss: 0.6389440894126892
Epoch: 0, Loss: 0.6849325895309448
Epoch: 0, Loss: 0.7703819274902344
Epoch: 0, Loss: 0.7393357753753662
Epoch: 0, Loss: 0.7198930978775024
Epoch: 0, Loss: 0.71733158826828
Epoch: 0, Loss: 0.7702065110206604
Epoch: 0, Loss: 0.7557789087295532
Epoch: 0, Loss: 0.7466634511947632
Epoch: 0, Loss: 0.6430736184120178
Epoch: 0, Loss: 0.751858115196228
Epoch: 0, Loss: 0.7444820404052734
Epoch: 0, Loss: 0.7069908976554871
Epoch: 0, Loss: 0.7426291704177856
Epoch: 0, Loss: 0.6321664452552795
Epoch: 0, Loss: 0.6869156956672668
Epoch: 0, Loss: 0.7027770280838013
Epoch: 0, Loss: 0.6982196569442749
Epoch: 0, Loss: 0.593335747718811
Epoch: 0, Loss: 0.8333384394645691
Epoch: 0, Loss: 0.6891831755638123
Epoch: 0, Loss: 0.6389833688735962
Epoch: 0, Loss: 0.5162726044654846
Epoch: 0, Loss: 0.6661826968193054
Epoch: 0, Loss: 0.7197288870811462
Epoch: 0, Loss: 0.6603603363037109
Epoch: 0, Loss: 0.622469

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

overall_accuracies, accuracies, f1s, precisions, recalls = [], [], [], [], []

# for run in range(1):
for run in range(3):
  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  print(model_path)

  model = mpc.from_pretrained(model_base, num_labels = 2)
  model.load_state_dict(torch.load(model_path))

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  tokenizer = tkz.from_pretrained(model_base)
  validation_dataset = CladderDataset(validation_data, tokenizer)
  validation_dataloader = DataLoader(validation_dataset, batch_size=8)




  model.eval()
  correct_predictions = 0
  total_predictions = 0

  accuracy, f1, precision, recall = [], [], [], []
  batches = 0

  def compute_metrics(y_true, y_pred):
      accuracy = accuracy_score(y_true, y_pred)
      # print(y_true)
      # print(y_pred)
      f1 = f1_score(y_true, y_pred)

      precision = precision_score(y_true, y_pred)
      # print(precision)
      recall = recall_score(y_true, y_pred)
      return [accuracy, f1, precision, recall]

  with torch.no_grad():
      for batch in validation_dataloader:
          input_ids = batch['input_ids'].squeeze(1)
          attention_mask = batch['attention_mask'].squeeze(1)
          labels = batch['labels'].cpu()  # ground truth
          # print(labels)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          # print(outputs)
          # print(outputs.logits)
          predictions = torch.argmax(outputs.logits, dim=1).cpu()  # predictions
          # print(predictions)

          calculated_metrics = compute_metrics(labels, predictions)
          accuracy.append(calculated_metrics[0])
          f1.append(calculated_metrics[1])
          precision.append(calculated_metrics[2])
          recall.append(calculated_metrics[3])
          batches += 1

          correct_predictions += (predictions == labels).sum().item()
          total_predictions += labels.size(0)

  overall_accuracy = correct_predictions / total_predictions
  print(f"Overall Accuracy: {overall_accuracy:.4f}")
  overall_accuracies.append(overall_accuracy)

  accuracy_var = sum(accuracy)/batches
  print(f"Accuracy: {sum(accuracy)/batches:.4f}")
  print(accuracy_var)
  accuracies.append(accuracy_var)

  f1_var = sum(f1)/batches
  print(f"F1: {sum(f1)/batches:.4f}")
  print(f1_var)
  f1s.append(f1_var)

  precision_var = sum(precision)/batches
  print(f"Precision: {sum(precision)/batches:.4f}")
  print(precision_var)
  precisions.append(precision_var)

  recall_var = sum(recall)/batches
  print(f"Recall: {sum(recall)/batches:.4f}")
  print(recall_var)
  recalls.append(precision_var)

  print(f'Number of batches: {batches}')

cladder_albert-base-v2_0.pt


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.4905
Accuracy: 0.4905
0.49050632911392406
F1: 0.6387
0.6386568424543108
Precision: 0.4905
0.49050632911392406
Recall: 0.9968
0.9968354430379747
Number of batches: 316
cladder_albert-base-v2_1.pt


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defi

Overall Accuracy: 0.5036
Accuracy: 0.5036
0.5035601265822784
F1: 0.1086
0.10858700020092416
Precision: 0.2363
0.23628691983122366
Recall: 0.0745
0.07452531645569614
Number of batches: 316
cladder_albert-base-v2_2.pt


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.4905
Accuracy: 0.4905
0.49050632911392406
F1: 0.6387
0.6386568424543108
Precision: 0.4905
0.49050632911392406
Recall: 0.9968
0.9968354430379747
Number of batches: 316


In [12]:
# # Prepare the DataLoader
# cladder_dataset = CladderDataset(train_data, tokenizer)
# dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

# # Optimizer and Loss Function
# optimizer = optim.Adam(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
# # Training Loop
# model.train()

# for epoch in range(3):  # Number of epochs
#   for batch in dataloader:
#       optimizer.zero_grad()

#       input_ids = batch['input_ids'].squeeze(1)
#       attention_mask = batch['attention_mask'].squeeze(1)
#       labels = batch['labels']

#       outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs.loss

#       loss.backward()
#       optimizer.step()

#       print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [14]:
# validation_dataset = CladderDataset(validation_data, tokenizer)
# validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        print(f"predictions: {predictions}, labels: {labels}")
        total_predictions += labels.size(0)

print(f"total_predictions: {total_predictions}")

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


predictions: tensor([0, 0, 0, 1, 0, 0, 1, 1], device='cuda:0'), labels: tensor([1, 0, 0, 1, 0, 1, 0, 1], device='cuda:0')
predictions: tensor([0, 1, 1, 0, 0, 0, 0, 1], device='cuda:0'), labels: tensor([1, 0, 1, 0, 0, 0, 0, 1], device='cuda:0')
predictions: tensor([1, 0, 1, 1, 0, 0, 1, 0], device='cuda:0'), labels: tensor([1, 1, 0, 1, 0, 1, 1, 1], device='cuda:0')
predictions: tensor([0, 0, 1, 0, 1, 0, 1, 0], device='cuda:0'), labels: tensor([0, 1, 1, 1, 1, 1, 0, 0], device='cuda:0')
predictions: tensor([1, 0, 0, 1, 1, 1, 0, 0], device='cuda:0'), labels: tensor([0, 1, 1, 1, 0, 0, 1, 0], device='cuda:0')
predictions: tensor([0, 1, 1, 0, 1, 0, 0, 0], device='cuda:0'), labels: tensor([1, 0, 0, 1, 1, 1, 0, 1], device='cuda:0')
predictions: tensor([0, 1, 1, 1, 0, 0, 1, 0], device='cuda:0'), labels: tensor([1, 1, 0, 1, 0, 0, 0, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 0, 1, 1, 1, 0], device='cuda:0'), labels: tensor([0, 1, 0, 0, 1, 0, 1, 0], device='cuda:0')
predictions: tensor([1, 